In [ ]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

from torchsummary import summary as summary
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, random_split



import numpy as np


In [ ]:
class Local_Discriminator2(nn.Module):
    def __init__(self):
        super(Local_Discriminator2, self).__init__()
        
        self.nc = 3
        self.ndf = 32
        
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
class Local_Discriminator(nn.Module):
    def __init__(self):
        super(Local_Discriminator, self).__init__()
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 1, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
#LOADING SAMPLE DATA

data_dir = "C:\\Users\\Sean\\Desktop\\UCSD Courses\\ECE228 Project\\CELEBA_NOSE\\sample"

p =transforms.Compose([transforms.ToTensor()])

dataset = datasets.ImageFolder(root = data_dir, transform = p)
print(dataset.class_to_idx)

idx2class = {v: k for k, v in hotdog_dataset.class_to_idx.items()}
print(idx2class)

In [ ]:
# SPLIT INTO VALIDATION AND TEST SET USING random_split

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train, valid = torch.utils.data.random_split(dataset, [train_size, test_size])

train_dl = DataLoader(train, batch_size=32, shuffle=True)
valid_dl = DataLoader(valid, batch_size=1024, shuffle=False)

#SET OPTIMIZER, CRITERION ETC
model = Local_Discriminator2()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.008)


In [ ]:
#SAMPLE FOR DATALOADER. NOTE VALIDATION AND TEST NOT INCLUDED

losses = []

for epoch in range(10):
    for i, (inputs, targets) in enumerate(train_dl):
        print("EPOCH ", i)
        
        running_loss = 0.0
        
        optimizer.zero_grad()
        yhat = model(inputs)
        yhat= torch.flatten(yhat) #IMPORTANT
        loss = criterion(yhat, targets.float()) #IMPORTANT
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        
    epoch_loss = running_loss / len(train_dl)
    print(epoch_loss)
    losses.append(epoch_loss)